In [153]:
import pandas as pd
import datetime
import requests as req
import numpy as np

In [57]:
path = r"\\home.rrze.uni-erlangen.de\douk11ibag\Documents\Python\big data\athlete_events.csv"

In [65]:
df = pd.read_csv(path)
print(len(df))
df = df[df['Year'] >= 1980]

271116


# Target

NOC merge

Games Title erstellen -> 1900 Summergames // Zusatz 1900 Summer Paris

# Year Season City
merge with two fields

In [68]:
year_season_city = df[['Year', 'Season', 'City']]
year_season_city = year_season_city.drop_duplicates()
year_season_city = year_season_city.sort_values(['Year'], ascending=False)
year_season_city = year_season_city.reset_index(drop=True)
year_season_city.sample(4)

,Year,Season,City
17,1984,Summer,Los Angeles
9,1998,Winter,Nagano
7,2002,Winter,Salt Lake City
14,1988,Winter,Calgary


In [60]:
year_season_city.to_csv(r"\\home.rrze.uni-erlangen.de\douk11ibag\Documents\Python\big data\year_season_city.csv")

# Noc
left noc and right noc_id, some nocs are missing -> left join!

In [61]:
pd.merge()

TypeError: merge() missing 2 required positional arguments: 'left' and 'right'

# Athlets
left on ID, right on Index

In [64]:
athleths = df[['ID', 'Name', 'Sex', 'Age', 'Height', 'Weight']]
athleths = athleths.drop_duplicates(['ID'])
athleths = athleths.set_index('ID',drop=True)
athleths['Name'].str.split(' ', n=1)

ID
1                                    [A, Dijiang]
2                                     [A, Lamusi]
5                     [Christine, Jacoba Aaftink]
6                              [Per, Knut Aaland]
7                                 [John, Aalberg]
9                             [Antti, Sami Aalto]
11                          [Jorma, Ilmari Aalto]
12                           [Jyri, Tapani Aalto]
13                          [Minna, Maarit Aalto]
14              [Pirjo, Hannele Aalto (Mattila-)]
16                    [Juhamatti, Tapio Aaltonen]
18                        [Timo, Antero Aaltonen]
20                          [Kjetil, Andr Aamodt]
21                   [Ragnhild, Margrethe Aamodt]
22                               [Andreea, Aanei]
23                                 [Fritz, Aanes]
26                [Agnes, Erika Aanonsen (-Eyde)]
27                               [Johan, Aantjes]
29                        [Willemien, Aardenburg]
30                            [Pepijn, Aardewij

# Etherium Preise

In [188]:
resp = req.get('https://query1.finance.yahoo.com/v8/finance/chart/BTC-EUR?symbol=BTC-EUR&period1=1557342840&period2=1557580689&interval=15m&includePrePost=true&events=div|split|earn&lang=de-DE&region=DE&crumb=NFWlGk6OV6P&corsDomain=de.finance.yahoo.com')

d = resp.json()
dt = d['chart']['result'][0]['timestamp']
df_stock = {'timestamp':dt}

for k in d['chart']['result'][0]['indicators']['quote'][0].keys():
    df_stock[k] = d['chart']['result'][0]['indicators']['quote'][0][k]
    
df_stock = pd.DataFrame(df_stock)
min15 = 60*15
df_stock['timestamp'] = df_stock['timestamp'] + (min15 - np.random.random(len(df_stock)) * min15 * 2).astype(int)
df_stock['timestamp'] = pd.to_datetime(df_stock['timestamp'], unit='s')
df_stock = df_stock.sort_values('timestamp')
df_stock.sample(10)

,timestamp,open,low,close,high,volume
144,2019-05-10 07:03:13,5615.450195,5602.740234,5611.149902,5635.459961,1400945
96,2019-05-09 18:49:12,5408.750000,5402.089844,5411.549805,5411.549805,536288
262,2019-05-11 12:40:36,6023.600098,6007.819824,6007.839844,6025.779785,602224
255,2019-05-11 10:46:57,6046.910156,6042.410156,6083.430176,6085.129883,1055156
149,2019-05-10 08:09:55,5633.100098,5610.799805,5614.680176,5633.100098,1702082
42,2019-05-09 05:29:44,5415.029785,5413.370117,5416.220215,5426.299805,366811
88,2019-05-09 16:49:35,5406.290039,5397.879883,5405.660156,5410.759766,857088
236,2019-05-11 06:02:10,5977.750000,5976.209961,6092.810059,6114.870117,3260422
111,2019-05-09 22:46:40,5502.540039,5484.220215,5497.919922,5505.750000,608708
192,2019-05-10 18:46:01,5709.370117,5686.439941,5691.859863,5711.759766,1369288


In [175]:
dtu = df_stock['timestamp'].dt.date.unique()

In [176]:
df_open = pd.DataFrame({'Date':dtu})
df_open['key'] = 0
df_open['Location'] = 'FRA'

df_opening_times = pd.DataFrame({'Time': [8, 22]})
df_opening_times['key'] = 0

df_market = pd.merge(df_open, df_opening_times, on='key')
df_market = df_market.drop(columns=['key'])

df_market['Date'] = df_market['Date'] + df_market['Time'].apply(lambda x: pd.DateOffset(hours=x))

In [179]:
# solution
df_solution = pd.merge_asof(df_market, df_stock, left_on='Date', right_on='timestamp', direction='nearest')
df_solution

,Date,Location,Time,timestamp,volume,close,open,high,low
0,2019-05-08 08:00:00,FRA,8,2019-05-08 18:50:04,0,5272.580078,5273.229980,5274.000000,5272.459961
1,2019-05-08 22:00:00,FRA,22,2019-05-08 22:04:40,276932,5280.270020,5289.560059,5292.149902,5280.120117
2,2019-05-09 08:00:00,FRA,8,2019-05-09 08:05:45,558388,5442.979980,5428.810059,5444.540039,5428.810059
3,2019-05-09 22:00:00,FRA,22,2019-05-09 21:57:41,991024,5482.720215,5464.439941,5484.129883,5458.529785
4,2019-05-10 08:00:00,FRA,8,2019-05-10 07:57:47,1909982,5632.410156,5642.930176,5656.069824,5620.810059
5,2019-05-10 22:00:00,FRA,22,2019-05-10 22:05:59,379928,5667.979980,5651.529785,5670.750000,5651.529785
6,2019-05-11 08:00:00,FRA,8,2019-05-11 07:54:44,2288096,6017.609863,6052.459961,6062.890137,6016.379883
7,2019-05-11 22:00:00,FRA,22,2019-05-11 13:04:57,86224,6044.529785,6049.640137,6049.890137,6043.830078


In [186]:
# median columnwise
df_solution['median'] = df_solution[['open', 'close']].median(axis=1)
df_solution

,Date,Location,Time,timestamp,volume,close,open,high,low,median
0,2019-05-08 08:00:00,FRA,8,2019-05-08 18:50:04,0,5272.580078,5273.229980,5274.000000,5272.459961,5272.905029
1,2019-05-08 22:00:00,FRA,22,2019-05-08 22:04:40,276932,5280.270020,5289.560059,5292.149902,5280.120117,5284.915039
2,2019-05-09 08:00:00,FRA,8,2019-05-09 08:05:45,558388,5442.979980,5428.810059,5444.540039,5428.810059,5435.895020
3,2019-05-09 22:00:00,FRA,22,2019-05-09 21:57:41,991024,5482.720215,5464.439941,5484.129883,5458.529785,5473.580078
4,2019-05-10 08:00:00,FRA,8,2019-05-10 07:57:47,1909982,5632.410156,5642.930176,5656.069824,5620.810059,5637.670166
5,2019-05-10 22:00:00,FRA,22,2019-05-10 22:05:59,379928,5667.979980,5651.529785,5670.750000,5651.529785,5659.754883
6,2019-05-11 08:00:00,FRA,8,2019-05-11 07:54:44,2288096,6017.609863,6052.459961,6062.890137,6016.379883,6035.034912
7,2019-05-11 22:00:00,FRA,22,2019-05-11 13:04:57,86224,6044.529785,6049.640137,6049.890137,6043.830078,6047.084961


In [185]:
# prüfsumme
df_solution['median'].sum()

44846.840087890625